## Practical 1: word2vec
Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [20]:
# My Edit 1, finding the top 1000 occuring words
counts_ted_top1000 = []
from collections import Counter
c = Counter()
for sentence in sentences_ted:
    c += Counter(sentence)
top1000WordCounts = c.most_common(1000) # this is a list of tuples: [(word, count),...]
counts_ted_top1000 = [x[1] for x in top1000WordCounts]

KeyboardInterrupt: 

In [21]:
allwords = []
with open('allwords.txt', 'w') as out:
    # lay the counter into a plain list, then sort by their count value
    for k in c.keys():
        allwords.append((k, c[k])) #allwords stores the tuples (word, count)
    allwords.sort(key = lambda x: (-x[1], x[0]))

In [22]:
print (len(allwords))
print (allwords[:10])

10504
[('the', 5885), ('and', 4163), ('to', 3649), ('of', 3261), ('a', 3081), ('that', 2709), ('we', 2258), ('in', 2196), ('i', 2177), ('it', 1984)]


In [23]:
with open('allwords', 'w') as outfile:
    for tup in allwords:
        outfile.write(tup[0] + ' ' + str(tup[1]) + '\n')
        outfile.flush()

Plot distribution of top-1000 words

In [17]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [24]:
from gensim.models import Word2Vec

In [25]:
# my code edit 2

# initialize a model with min_count = 10
model_ted = Word2Vec(sentences_ted,min_count = 0)

In [26]:
print (len(model_ted.vocab))
#save the model
model_ted.save('ted_model')


54454


### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [41]:
model_ted.most_similar("man")

[('woman', 0.8510145545005798),
 ('guy', 0.8070447444915771),
 ('lady', 0.770460844039917),
 ('girl', 0.7613636255264282),
 ('boy', 0.7463864684104919),
 ('soldier', 0.7284667491912842),
 ('gentleman', 0.7011345624923706),
 ('kid', 0.6866320371627808),
 ('poet', 0.6641428470611572),
 ('philosopher', 0.6617022752761841)]

In [42]:
model_ted.most_similar("computer")

[('machine', 0.7647178173065186),
 ('software', 0.7128764390945435),
 ('robot', 0.7085899114608765),
 ('3d', 0.6907616853713989),
 ('device', 0.67992103099823),
 ('program', 0.658528745174408),
 ('chip', 0.6585046648979187),
 ('camera', 0.6582038998603821),
 ('video', 0.6315593719482422),
 ('printer', 0.630183756351471)]

In [56]:
# my edit: alternative words with interesting nearest neighbors
model_ted.most_similar("language")

[('mathematics', 0.6643838882446289),
 ('culture', 0.659943163394928),
 ('english', 0.6539066433906555),
 ('logic', 0.6274464130401611),
 ('narrative', 0.6246222257614136),
 ('nature', 0.6029856204986572),
 ('domain', 0.5986967086791992),
 ('desire', 0.592326283454895),
 ('meaning', 0.588565468788147),
 ('empathy', 0.5859334468841553)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [44]:
# my edit
words_top_ted = [tup[0] for tup in top1000WordCounts]

In [45]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [46]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [47]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [48]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [49]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [50]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [51]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [76]:
print (sentences_wiki[:10])

[' first mlb pitcher in a world series to pitch a shutout with no walks and at least eight strikeouts   game   in      ', ' later that year   the band performed it at the girls aloud party tv special held by itv    and at the v festival ', ' references to horned rabbits may originate in sightings of rabbits affected by the shope papilloma virus   named for richard e', ' the      cleveland browns season was the team  s third in the all     america football conference   aafc   ', ' he suffered mild inflammation in his right shoulder in the arizona fall league after the conclusion of the      regular season ', ' precipitation has generally increased over land north of      n from      to        but declined over the tropics since the     s ', ' in addition   bonnie began to slowly move more to the northeast on september    as steering currents slowly developed ', ' the mysore lancers had twenty     three men killed in action   one man died as a result of his wounds   another two were repo

In [77]:
sentences_strings_wiki = sentences_wiki
sentences_wiki = []
for sent_str in sentences_strings_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki.append(tokens)

In [78]:
print (sentences_wiki[:10])

[['first', 'mlb', 'pitcher', 'in', 'a', 'world', 'series', 'to', 'pitch', 'a', 'shutout', 'with', 'no', 'walks', 'and', 'at', 'least', 'eight', 'strikeouts', 'game', 'in'], ['later', 'that', 'year', 'the', 'band', 'performed', 'it', 'at', 'the', 'girls', 'aloud', 'party', 'tv', 'special', 'held', 'by', 'itv', 'and', 'at', 'the', 'v', 'festival'], ['references', 'to', 'horned', 'rabbits', 'may', 'originate', 'in', 'sightings', 'of', 'rabbits', 'affected', 'by', 'the', 'shope', 'papilloma', 'virus', 'named', 'for', 'richard', 'e'], ['the', 'cleveland', 'browns', 'season', 'was', 'the', 'team', 's', 'third', 'in', 'the', 'all', 'america', 'football', 'conference', 'aafc'], ['he', 'suffered', 'mild', 'inflammation', 'in', 'his', 'right', 'shoulder', 'in', 'the', 'arizona', 'fall', 'league', 'after', 'the', 'conclusion', 'of', 'the', 'regular', 'season'], ['precipitation', 'has', 'generally', 'increased', 'over', 'land', 'north', 'of', 'n', 'from', 'to', 'but', 'declined', 'over', 'the', 't

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [87]:
# my code edit
# copied from My Edit 1, finding the top 1000 occuring words
counts_wiki_top1000 = []
from collections import Counter
c_wiki = Counter()
for sentence in sentences_wiki:
    c_wiki += Counter(sentence)
top1000WordCounts_wiki = c_wiki.most_common(1000) # this is a list of tuples: [(word, count),...]
counts_wiki_top1000 = [x[1] for x in top1000WordCounts_wiki]
words_top_wiki = [x[0] for x in top1000WordCounts_wiki]

In [94]:
#save the words to file
allwords_wiki = []
# lay the counter into a plain list, then sort by their count value
for k in c_wiki.keys():
    allwords_wiki.append((k, c_wiki[k])) #allwords stores the tuples (word, count)
allwords_wiki.sort(key = lambda x: (-x[1], x[0]))
with open ('allwords_wiki', 'w') as outfile_wiki:
    for tup in allwords_wiki:
        outfile_wiki.write(tup[0] + ' ' + str(tup[1]) + '\n')
        outfile_wiki.flush()

In [88]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p_wiki = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution Wiki")
p_wiki.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p_wiki)

In [79]:
# model the wiki data
# copied from my code edit 2

# initialize a model with min_count = 10
model_wiki = Word2Vec(sentences_wiki,min_count = 10)
model_wiki.save('wiki_model')

/home/scratch/donhan/anaconda3/lib/python3.5/site-packages/gensim/models/word2vec.py:761: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


In [84]:
print (model_wiki.most_similar('computer'))
print (model_wiki.most_similar('man'))

[('software', 0.7993074059486389), ('computers', 0.789939284324646), ('simulation', 0.7373844385147095), ('hardware', 0.7305177450180054), ('interface', 0.727923572063446), ('technology', 0.7196988463401794), ('user', 0.7079048156738281), ('analog', 0.7042402029037476), ('computing', 0.6918647289276123), ('virtual', 0.6771758794784546)]
[('woman', 0.671110987663269), ('boy', 0.6100094318389893), ('dog', 0.5843208432197571), ('girl', 0.5826656818389893), ('person', 0.5783488154411316), ('mask', 0.5756555795669556), ('soldier', 0.5672560930252075), ('creature', 0.5654325485229492), ('warrior', 0.547448992729187), ('thief', 0.545087456703186)]


#### t-SNE visualization

In [92]:
# debugging: words_top_wiki
words_top_wiki[:10]

['the', 'of', 'and', 'in', 'to', 'a', 'was', 's', 'on', 'as']

In [91]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [93]:
p3 = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p3.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p3.add_layout(labels)

show(p3)

In [95]:
# find some similar words
print(model_wiki.most_similar('computer'))

[('software', 0.7993074059486389), ('computers', 0.789939284324646), ('simulation', 0.7373844385147095), ('hardware', 0.7305177450180054), ('interface', 0.727923572063446), ('technology', 0.7196988463401794), ('user', 0.7079048156738281), ('analog', 0.7042402029037476), ('computing', 0.6918647289276123), ('virtual', 0.6771758794784546)]


In [96]:
print(model_wiki.most_similar('man'))

[('woman', 0.671110987663269), ('boy', 0.6100094318389893), ('dog', 0.5843208432197571), ('girl', 0.5826656818389893), ('person', 0.5783488154411316), ('mask', 0.5756555795669556), ('soldier', 0.5672560930252075), ('creature', 0.5654325485229492), ('warrior', 0.547448992729187), ('thief', 0.545087456703186)]
